# DeepFakes: Reenactment using Pix2Pix GAN
- We will leverage a live feed as input
- The live video feed would be used to extract facial landmarks
- The landmarks would then be used to generate reenacted frames of Mr. Barak Obama

## Load Libraries

In [ ]:
import cv2
import dlib
import numpy as np
from imutils import video
from matplotlib import pyplot as plt
from tensorflow import keras

In [ ]:
CROP_SIZE = 256
DOWNSAMPLE_RATIO = 4

## Utility Methods

In [ ]:
def reshape_for_polyline(array):
    """Reshape image so that it works with polyline."""
    return np.array(array, np.int32).reshape((-1, 1, 2))


def resize(image):
    """Crop and resize image for pix2pix."""
    height, width, _ = image.shape
    if height != width:
        # crop to correct ratio
        size = min(height, width)
        oh = (height - size) // 2
        ow = (width - size) // 2
        cropped_image = image[oh:(oh + size), ow:(ow + size)]
        image_resize = cv2.resize(cropped_image, (CROP_SIZE, CROP_SIZE), interpolation = cv2.INTER_LINEAR)
        return image_resize

def rescale_frame(frame):
    dim = (256, 256)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

## Load Models

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r'shape_predictor_68_face_landmarks.dat')

In [ ]:
generator = keras.models.load_model(r'gen_pix2pix_deepfake_obama.h5')

## Quick Sanity Check

In [ ]:
from data_utils import plot_sample_images
import tensorflow as tf
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (8,8),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}

plt.rcParams.update(params)

In [ ]:
plot_sample_images(generator,'data/')

## Utility Method for Live Feed

In [ ]:
def get_landmarks(black_image,gray,faces):
    for face in faces:
        detected_landmarks = predictor(gray, face).parts()
        landmarks = [[p.x * DOWNSAMPLE_RATIO, p.y * DOWNSAMPLE_RATIO] for p in detected_landmarks]

        jaw = reshape_for_polyline(landmarks[0:17])
        left_eyebrow = reshape_for_polyline(landmarks[22:27])
        right_eyebrow = reshape_for_polyline(landmarks[17:22])
        nose_bridge = reshape_for_polyline(landmarks[27:31])
        lower_nose = reshape_for_polyline(landmarks[30:35])
        left_eye = reshape_for_polyline(landmarks[42:48])
        right_eye = reshape_for_polyline(landmarks[36:42])
        outer_lip = reshape_for_polyline(landmarks[48:60])
        inner_lip = reshape_for_polyline(landmarks[60:68])

        color = (255, 255, 255)
        thickness = 3

        cv2.polylines(black_image, [jaw], False, color, thickness)
        cv2.polylines(black_image, [left_eyebrow], False, color, thickness)
        cv2.polylines(black_image, [right_eyebrow], False, color, thickness)
        cv2.polylines(black_image, [nose_bridge], False, color, thickness)
        cv2.polylines(black_image, [lower_nose], True, color, thickness)
        cv2.polylines(black_image, [left_eye], True, color, thickness)
        cv2.polylines(black_image, [right_eye], True, color, thickness)
        cv2.polylines(black_image, [outer_lip], True, color, thickness)
        cv2.polylines(black_image, [inner_lip], True, color, thickness)
    return black_image

In [ ]:
def rescale_frame(frame):
    dim = (256, 256)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [ ]:
def get_obama(landmarks):
    landmarks = (landmarks/127.5)-1
    landmarks = tf.image.resize(landmarks, [256,256]).numpy()
    fake_imgs = generator.predict(np.expand_dims(landmarks,axis=0))
    return fake_imgs

## Let's Start Faking

In [ ]:
cap = cv2.VideoCapture(0)
fps = video.FPS().start()
k = 0
display_plots = True
display_cv2 = True
while True:
    k += 1
    ret, frame = cap.read(0)
    if np.all(np.array(frame.shape)):
        frame_resize = cv2.resize(frame, None, fx=1 / DOWNSAMPLE_RATIO, fy=1 / DOWNSAMPLE_RATIO)
        gray = cv2.cvtColor(frame_resize, cv2.COLOR_BGR2GRAY)
        faces = detector(gray, 1)
        black_image = np.zeros(frame.shape, np.uint8)
        landmarks = get_landmarks(black_image.copy(),gray,faces)
        img_tgt = (landmarks/127.5)-1
        img_tgt = tf.image.resize(img_tgt, [256,256]).numpy()
        obama = generator.predict(np.expand_dims(img_tgt,axis=0))[0]
        try:
            obama = 0.5 * obama + 0.5
            gen_imgs = np.concatenate([np.expand_dims(cv2.cvtColor(rescale_frame(frame_resize), cv2.COLOR_RGB2BGR),axis=0), 
                                       np.expand_dims(rescale_frame(obama),axis=0), 
                                       np.expand_dims(rescale_frame(landmarks),axis=0)])
            if display_plots:
                titles = ['Live', 'Generated', 'Landmarks']
                rows, cols = 1, 3
                fig, axs = plt.subplots(rows, cols)
                for j in range(cols):
                    if j!=1:
                        axs[j].imshow(gen_imgs[j].astype(int))
                    else:
                        axs[j].imshow(gen_imgs[j])
                    axs[j].set_title(titles[j])
                    axs[j].axis('off')
                plt.show()
            if display_cv2:
                cv2.imshow('synthetic obama', cv2.cvtColor(gen_imgs[1], cv2.COLOR_BGR2RGB))
        except Exception as ex:
            print(ex)
        fps.update()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

fps.stop()
print('[INFO] elapsed time (total): {:.2f}'.format(fps.elapsed()))
print('[INFO] approx. FPS: {:.2f}'.format(fps.fps()))

#sess.close()
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()

## Sample Frame

In [ ]:
z = np.concatenate([gen_imgs[0].astype(int),
                    (255*gen_imgs[1]).astype(int),
                    gen_imgs[2].astype(int)],axis=1)
plt.imshow(z)
plt.axis('off')